In [ ]:
from bandit_functions import *
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np

In [ ]:
# Output folder for saving generated figures
out_folder_name = 'figures'

### Running simulation, where output is used later for visualization

In [ ]:
# =====================
# Simulation: Well-Specified Setting
# =====================

# Settings for the simulation
x_grid = np.arange(0, 251, 50)  # Fertilizer rates (arms) to consider
reps = 10  # Number of replicates for averaging results
T = 30  # Number of rounds (seasons)
p_y = 5.00  # Price of yield (e.g., $/unit)
noise_sd = 0.5  # Standard deviation of noise in yield
px_values = [0.3, 0.5, 0.7]  # Different fertilizer price scenarios

# Define models and their parameters (true and initial guesses)
models = {
    "mitscherlich": {
        "params_true": dict(A=120, b=0.015, d=80),  # True params for Mitscherlich model
        "params0": dict(A=100, b=0.01, d=75),       # Initial guess for Mitscherlich
    },
    "quadratic_plateau": {
        "params_true": dict(a=80, b=1.2, c=-0.003, x0=180),
        "params0": dict(a=75, b=1.0, c=-0.002, x0=160),
    },
    "michaelis_menten": {
        "params_true": dict(a=150, b=100, d=60),
        "params0": dict(a=120, b=80, d=50),
    },
    "logistic": {
        "params_true": dict(A=120, B=0.05, C=125, d=70),
        "params0": dict(A=100, B=0.03, C=100, d=65),
    }
}

# Initialize dictionaries to store simulation results for each price and model
regret_data = {px: {model: {} for model in models} for px in px_values}
arm_data = {px: {model: {} for model in models} for px in px_values}
profit_data = {px: {model: {} for model in models} for px in px_values}
param_data = {px: {model: {} for model in models} for px in px_values}
random_test_data = {px: {model: {} for model in models} for px in px_values}

# Run simulations for each fertilizer price and model
for p_x in px_values:
    print(f"Running px = {p_x}...")  # Indicate which price is being simulated
    for model_name, param_sets in models.items():
        print(f"  Model: {model_name}")  # Indicate which model is being simulated

        # --- Epsilon-Greedy Bandit Algorithm ---
        # Run simulation with epsilon-greedy strategy; returns various statistics
        avg_profit_eps, avg_cumprofit_eps, avg_regret_eps, avg_arm_eps, all_profits_eps, all_regrets_eps, _, avg_param_eps = run_bandit_replicates(
            epsilon_greedy_bandit,
            model=model_name,
            params_true=param_sets["params_true"],
            params0=param_sets["params0"],
            x_grid=x_grid, T=T,
            p_y=p_y, p_x=p_x,
            noise_sd=noise_sd, reps=reps,
            epsilon_sched=lambda t: t**(-1.5)  # Decaying epsilon schedule
        )

        # --- UCB Bandit Algorithm ---
        # Run simulation with Upper Confidence Bound strategy
        avg_profit_ucb, avg_cumprofit_ucb, avg_regret_ucb, avg_arm_ucb, all_profits_ucb, all_regrets_ucb, _, avg_param_ucb = run_bandit_replicates(
            ucb_bandit,
            model=model_name,
            params_true=param_sets["params_true"],
            params0=param_sets["params0"],
            x_grid=x_grid, T=T,
            p_y=p_y, p_x=p_x,
            noise_sd=noise_sd, reps=reps,
            alpha=0.02  # Exploration parameter for UCB
        )

        # --- ViOlin Bandit Algorithm ---
        # Run simulation with ViOlin strategy (curvature-based)
        avg_profit_violin, avg_cumprofit_violin, avg_regret_violin, avg_arm_violin, all_profits_violin, all_regrets_violin, _, avg_param_violin = run_bandit_replicates(
            violin_bandit_curvature,
            model=model_name,
            params_true=param_sets["params_true"],
            params0=param_sets["params0"],
            x_grid=x_grid, T=T,
            p_y=p_y, p_x=p_x,
            noise_sd=noise_sd, reps=reps,
        )

        # --- Store results in dictionaries for later visualization ---
        # Regret
        regret_data[p_x][model_name]["eps"] = avg_regret_eps  
        regret_data[p_x][model_name]["ucb"] = avg_regret_ucb  
        regret_data[p_x][model_name]["violin"] = avg_regret_violin 

        # Arm selection
        arm_data[p_x][model_name]["eps"] = avg_arm_eps  
        arm_data[p_x][model_name]["ucb"] = avg_arm_ucb 
        arm_data[p_x][model_name]["violin"] = avg_arm_violin 

        # Profit
        profit_data[p_x][model_name]["eps"] = avg_profit_eps   
        profit_data[p_x][model_name]["ucb"] = avg_profit_ucb  
        profit_data[p_x][model_name]["violin"] = avg_profit_violin  

        # Parameter estimates
        param_data[p_x][model_name]["eps"] = avg_param_eps  
        param_data[p_x][model_name]["ucb"] = avg_param_ucb  
        param_data[p_x][model_name]["violin"] = avg_param_violin  

    print("All simulations completed.") 

Running px = 0.3...
  Model: mitscherlich
  Model: quadratic_plateau
  Model: michaelis_menten
  Model: logistic
All simulations completed.
Running px = 0.5...
  Model: mitscherlich
  Model: quadratic_plateau
  Model: michaelis_menten
  Model: logistic
All simulations completed.
Running px = 0.7...
  Model: mitscherlich
  Model: quadratic_plateau
  Model: michaelis_menten
  Model: logistic
All simulations completed.


### Plotting cumulative regret

In [ ]:
# =====================
# Plotting: Cumulative Regret
# =====================

# For each model, plot the average cumulative regret for each algorithm and fertilizer price
for model_name in ["mitscherlich", "quadratic_plateau", "michaelis_menten", "logistic"]:

    colors = {"eps": "tab:blue", "ucb": "tab:orange", "violin": "tab:green"}  # Color for each algorithm
    linestyles = {"eps": "--", "ucb": "-", "violin": ":"}  # Line style for each algorithm
    labels = {
        "eps": "Epsilon-Greedy",
        "ucb": "UCB",
        "violin": "ViOlin"
    }

    for p_x in px_values:
        fig, ax = plt.subplots(figsize=(10, 6))
        fig.suptitle(f"Average Cumulative Regret — {model_name.replace('_', ' ').title()} — $p_x$ = {p_x}", fontsize=16, fontweight="bold")

        # Plot regret for each algorithm
        for alg in ["eps", "ucb", "violin"]:
            regret = regret_data[p_x][model_name][alg]
            ax.plot(
                regret,
                label=f"{labels[alg]}",
                color=colors[alg],
                linestyle=linestyles[alg],
                linewidth=3
            )

        ax.set_xlabel("Round")
        ax.set_ylabel("Avg Cumulative Regret ($)")
        ax.grid(True)
        ax.legend(fontsize=9, title='Bandit Algorithm')

        plt.tight_layout()
        # Save figures to different folders based on scenario
        if (p_x == 0.5) and (model_name == "quadratic_plateau"):
            out_dir = f"{out_folder_name}/well-specified"
            os.makedirs(out_dir, exist_ok=True)
            plt.savefig(f"{out_dir}/regret_{model_name}_px_{p_x}.png", dpi=300, bbox_inches='tight')
        else:
            out_dir = f"{out_folder_name}/appendix/well-specified/regret"
            os.makedirs(out_dir, exist_ok=True)
            plt.savefig(f"{out_dir}/regret_{model_name}_px_{p_x}.png", dpi=300, bbox_inches='tight')
        plt.close()
        # plt.show()

### Plotting running proportion

In [ ]:
# =====================
# Plotting: Running Proportion of Most Frequently Selected Fertilizer
# =====================

# Set up parameters for plotting
x_grid = np.arange(0, 251, 50)
n_arms = len(x_grid)
T = 30
reps = 30
px_values = [0.3, 0.5, 0.7]

color_map = {arm: plt.cm.tab10(i % 10) for i, arm in enumerate(x_grid)}  # Color for each arm
labels = {
        "eps": "Epsilon-Greedy",
        "ucb": "UCB",
        "violin": "ViOlin"
    }

# For each price and model, plot the running proportion for each algorithm
for p_x in px_values:
    for model_name in models.keys():
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        fig.suptitle(f"Running Proportion of Most Frequently Selected Fertilizer Over Time \n {model_name.replace('_', ' ').title()} — $p_x$ = {p_x}", fontsize=16, fontweight="bold")

        for ax, alg in zip(axs, ["eps", "ucb", "violin"]):
            chosen_arms = arm_data[p_x][model_name][alg]

            # Calculate average running proportion for each arm
            running_props = np.zeros((n_arms, T))
            for a_index, arm in enumerate(x_grid):
                running_props[a_index] = np.cumsum(chosen_arms == arm) / np.arange(1, T + 1)

            line_styles = ['-', '--', '-.', ':', (0, (3, 1, 1, 1)), (0, (5, 1))]

            # Plot running proportion for each arm
            for a_index, arm in enumerate(x_grid):
                label = f"{arm}"
                linestyle = line_styles[a_index % len(line_styles)]  # Cycle through line styles
                ax.plot(
                    running_props[a_index], 
                    label=label, 
                    linestyle=linestyle, 
                    color=color_map[arm], 
                    linewidth=2
                )
                
            ## Gradually decrease transparency so we can see overlapping lines
            # for a_index, arm in enumerate(x_grid):
            #     label = f"{arm}"
            #     alpha_value = 1 - (a_index / len(x_grid)) 
            #     ax.plot(
            #         running_props[a_index], 
            #         label=label, 
            #         linestyle=linestyle, 
            #         color=color_map[arm], 
            #         linewidth=3, 
            #         alpha=alpha_value
            #     )

            ax.set_title(f"{labels[alg]}", fontweight="bold")
            ax.set_xlabel("Round")
            ax.set_ylabel("Proportion selected up to round")
            ax.set_ylim(0, 1)
            ax.grid(True)
            ax.legend(title='Fertilizer Rate (lb N/ac)', fontsize=8, ncol=2)

        plt.tight_layout()
        # Save figures to different folders based on scenario
        if (p_x == 0.5) and (model_name == "quadratic_plateau"):
            out_dir = f"{out_folder_name}/well-specified"
            os.makedirs(out_dir, exist_ok=True)
            plt.savefig(f"{out_dir}/running_proportion_{model_name}_px_{p_x}.png", dpi=300, bbox_inches='tight')
        else:
            out_dir = f"{out_folder_name}/appendix/well-specified/running_proportion"
            os.makedirs(out_dir, exist_ok=True)
            plt.savefig(f"{out_dir}/running_proportion_{model_name}_px_{p_x}.png", dpi=300, bbox_inches='tight')
        plt.close()
        # plt.show()

### Plotting arm selection over time

In [ ]:
# =====================
# Plotting: Most Frequently Selected Fertilizer Rate Over Time
# =====================

# Set up parameters for plotting
x_grid = np.arange(0, 251, 50)
px_values = [0.3, 0.5, 0.7]
colors = {"eps": "tab:blue", "ucb": "tab:orange", "violin": "tab:green"}
linestyles = {"eps": "--", "ucb": "-", "violin": ":"}
labels = {"eps": "Epsilon-Greedy", "ucb": "UCB", "violin": "ViOlin"}

# For each model, plot the most frequently selected arm for each price and algorithm
for model_name in models.keys():
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f"Most Frequently Selected Fertilizer Rate Over Time By Price \n {model_name.replace('_', ' ').title()}", fontsize=16, fontweight="bold")

    for ax, p_x in zip(axs, px_values):
        for alg in ["eps", "ucb", "violin"]:
            arm = arm_data[p_x][model_name][alg]
            ax.plot(
                arm,
                label=f"{labels[alg]}",
                color=colors[alg],
                linestyle=linestyles[alg],
                linewidth=2
            )

        ax.set_title(f"$p_x$ = {p_x}", fontweight="bold")
        ax.set_xlabel("Round")
        ax.set_ylabel("Most Frequently Selected Fertilizer Rate")
        ax.set_ylim(0, arm_data[p_x][model_name]["eps"].max() + 10)
        ax.grid(True)
        ax.legend(title="Bandit Algorithm", fontsize=9)

    plt.tight_layout()
    # Save figures to different folders based on scenario
    if (model_name == "quadratic_plateau"):
        out_dir = f"{out_folder_name}/well-specified"
        os.makedirs(out_dir, exist_ok=True)
        plt.savefig(f"{out_dir}/mode_arm_{model_name}.png", dpi=300, bbox_inches='tight')
    else:
        out_dir = f"{out_folder_name}/appendix/well-specified/mode_arm"
        os.makedirs(out_dir, exist_ok=True)
        plt.savefig(f"{out_dir}/mode_arm_{model_name}.png", dpi=300, bbox_inches='tight')
    plt.close()
    # plt.show()

### Plotting Parameters over time

In [ ]:
# =====================
# Plotting: Parameters Over Time
# =====================

# For each price and model, plot the evolution of parameter estimates for each algorithm
px_values = [0.3, 0.5, 0.7]
labels = {"eps": "Epsilon-Greedy", "ucb": "UCB", "violin": "ViOlin"}

for p_x in px_values:
    for model_name, model_info in models.items():
        param_names = list(model_info["params0"].keys())  # Parameter names for the model
        num_params = len(param_names)

        fig, axs = plt.subplots(2, 2, figsize=(12, 10))
        axs = axs.ravel()

        fig.suptitle(f"Parameters Over Time — {model_name.replace('_', ' ').title()} — $p_x$ = {p_x}", fontsize=16, fontweight="bold")

        # Plot parameter evolution for each algorithm
        for alg, linestyle in zip(["eps", "ucb", "violin"], ["--", "-.", ":"]):
            param_array = param_data[p_x][model_name][alg]

            for i, param_name in enumerate(param_names):
                # Plot true parameter value as a horizontal line (only once per subplot)
                axs[i].axhline(model_info["params_true"][param_name], color="red", linestyle="-", linewidth=2, label="True" if alg == "eps" else "")
                axs[i].plot(
                    np.arange(1, 31), 
                    param_array[:, i],
                    label=f"{labels[alg]}",
                    linestyle=linestyle,
                    linewidth=2
                )
                axs[i].set_ylabel(param_name)
                axs[i].set_xlabel("Round")
                axs[i].grid(True)
        
        for ax in axs[:num_params]:
            ax.legend(title="Bandit Algorithm")

        plt.tight_layout()
        # Save figures to different folders based on scenario
        if (p_x == 0.5) and (model_name == "quadratic_plateau"):
            out_dir = f"{out_folder_name}/well-specified"
            os.makedirs(out_dir, exist_ok=True)
            plt.savefig(f"{out_dir}/params_{model_name}_px_{p_x}.png", dpi=300, bbox_inches='tight')
        else:
            out_dir = f"{out_folder_name}/appendix/well-specified/params"
            os.makedirs(out_dir, exist_ok=True)
            plt.savefig(f"{out_dir}/params_{model_name}_px_{p_x}.png", dpi=300, bbox_inches='tight')
        plt.close()
        # plt.show()

## Misspecified setting simulation

In [ ]:
from bandit_functions import *
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np

In [ ]:
# =====================
# Simulation: Misspecified Setting
# =====================

# Settings for the misspecified simulation
x_grid = np.arange(0, 251, 50)  # Fertilizer rates (arms) to consider
reps = 10  # Number of replicates for averaging results
T = 100  # Number of rounds (seasons)
p_y = 5.00  # Price of yield (e.g., $/unit)
noise_sd = 0.5  # Standard deviation of noise in yield
px_values = [0.3, 0.5, 0.7]  # Different fertilizer price scenarios

# Define models and their parameters (true and initial guesses)
models = {
    "mitscherlich": {
        "params_true": dict(A=120, b=0.015, d=80),  # True params for Mitscherlich model
        "params0": dict(A=100, b=0.01, d=75),       # Initial guess for Mitscherlich
    },
    "quadratic_plateau": {
        "params_true": dict(a=80, b=1.2, c=-0.003, x0=180),
        "params0": dict(a=75, b=1.0, c=-0.002, x0=160),
    },
}

# Initialize dictionaries to store simulation results
regret_data = {px: {model: {} for model in models} for px in px_values}
arm_data = {px: {model: {} for model in models} for px in px_values}
param_data = {px: {model: {} for model in models} for px in px_values}

# Run simulations for each fertilizer price and model (true/fit misspecified)
for p_x in px_values:
    print(f"Running px = {p_x}...")
    for i, (model_true_name, model_true_info) in enumerate(models.items()):
        for model_fit_name, model_fit_info in models.items():
            if model_true_name != model_fit_name:
                print(f"Running {model_true_name} with fit {model_fit_name}")

            # --- Epsilon-Greedy Bandit Algorithm (Misspecified) ---
            _, _, avg_regret_eps, avg_arm_eps, _, _, _, avg_param_eps = run_bandit_replicates_misspecified(
                epsilon_greedy_bandit_misspecified,
                model_true=model_true_name, params_true=model_true_info["params_true"],
                model_fit=model_fit_name, params0=model_fit_info["params0"],
                x_grid=x_grid, T=T, p_y=p_y, p_x=p_x, noise_sd=noise_sd, reps=reps, 
                epsilon_sched=lambda t: t**(-1.5)
            )

            # --- UCB Bandit Algorithm (Misspecified) ---
            _, _, avg_regret_ucb, avg_arm_ucb, _, _, _, avg_param_ucb = run_bandit_replicates_misspecified(
                ucb_bandit_misspecified,
                model_true=model_true_name, params_true=model_true_info["params_true"],
                model_fit=model_fit_name, params0=model_fit_info["params0"],
                x_grid=x_grid, T=T, p_y=p_y, p_x=p_x, noise_sd=noise_sd, reps=reps, 
                alpha=60
            )

            # --- ViOlin Bandit Algorithm (Misspecified) ---
            _, _, avg_regret_violin, avg_arm_violin, _, _, _, avg_param_violin = run_bandit_replicates_misspecified(
                violin_bandit_curvature_misspecified,
                model_true=model_true_name, params_true=model_true_info["params_true"],
                model_fit=model_fit_name, params0=model_fit_info["params0"],
                x_grid=x_grid, T=T, p_y=p_y, p_x=p_x, noise_sd=noise_sd, reps=reps,
                kappa_grad=5.0, kappa_hess=150.0 
            )
            
            # --- Store results in dictionaries for later visualization ---
            regret_data[p_x][model_fit_name]["eps"] = avg_regret_eps
            regret_data[p_x][model_fit_name]["ucb"] = avg_regret_ucb
            regret_data[p_x][model_fit_name]["violin"] = avg_regret_violin

            arm_data[p_x][model_fit_name]["eps"] = avg_arm_eps
            arm_data[p_x][model_fit_name]["ucb"] = avg_arm_ucb
            arm_data[p_x][model_fit_name]["violin"] = avg_arm_violin

            param_data[p_x][model_fit_name]["eps"] = avg_param_eps
            param_data[p_x][model_fit_name]["ucb"] = avg_param_ucb
            param_data[p_x][model_fit_name]["violin"] = avg_param_violin
        print("All simulations completed.")

Running px = 0.3...
Running mitscherlich with fit quadratic_plateau
All simulations completed.
Running quadratic_plateau with fit mitscherlich
All simulations completed.
Running px = 0.5...
Running mitscherlich with fit quadratic_plateau
All simulations completed.
Running quadratic_plateau with fit mitscherlich
All simulations completed.
Running px = 0.7...
Running mitscherlich with fit quadratic_plateau
All simulations completed.
Running quadratic_plateau with fit mitscherlich
All simulations completed.


### Plotting cumulative regret

In [ ]:
# =====================
# Plotting: Cumulative Regret (Misspecified Setting)
# =====================

# For each model, plot the average cumulative regret for each algorithm and fertilizer price in the misspecified setting
for model_name in ["mitscherlich", "quadratic_plateau"]:
    if model_name == "mitscherlich":
        true_name = "quadratic_plateau"
    else:
        true_name = "mitscherlich"

    colors = {"eps": "tab:blue", "ucb": "tab:orange", "violin": "tab:green"}  # Color for each algorithm
    linestyles = {"eps": "--", "ucb": "-", "violin": ":"}  # Line style for each algorithm
    labels = {
        "eps": "Epsilon-Greedy",
        "ucb": "UCB",
        "violin": "ViOlin"
    }

    for p_x in px_values:
        fig, ax = plt.subplots(figsize=(10, 6))
        true_name_cleaned = true_name.replace('_', ' ').title()
        fit_name_cleaned = model_name.replace('_', ' ').title()
        fig.suptitle(f"Average Cumulative Regret — $p_x$ = {p_x} \n True: {true_name_cleaned} - Fit: {fit_name_cleaned}", fontsize=16, fontweight="bold")

        # Plot regret for each algorithm
        for alg in ["eps", "ucb", "violin"]:
            regret = regret_data[p_x][model_name][alg]
            ax.plot(
                regret,
                label=f"{labels[alg]}",
                color=colors[alg],
                linestyle=linestyles[alg],
                linewidth=3
            )

        ax.set_xlabel("Round")
        ax.set_ylabel("Avg Cumulative Regret ($)")
        ax.grid(True)
        ax.legend(fontsize=9, title='Bandit Algorithm')

        plt.tight_layout()
        # Save figures to different folders based on scenario
        if (p_x == 0.5) and (model_name == "quadratic_plateau"):
            out_dir = f"{out_folder_name}/misspecified"
            os.makedirs(out_dir, exist_ok=True)
            plt.savefig(f"{out_dir}/regret_fit-{model_name}_true-{true_name}_px_{p_x}.png", dpi=300, bbox_inches='tight')
        else:
            out_dir = f"{out_folder_name}/appendix/misspecified/regret"
            os.makedirs(out_dir, exist_ok=True)
            plt.savefig(f"{out_dir}/regret_fit-{model_name}_true-{true_name}_px_{p_x}.png", dpi=300, bbox_inches='tight')
        plt.close()
        # plt.show()

### Plotting running proportion

In [ ]:
# =====================
# Plotting: Running Proportion of Most Frequently Selected Fertilizer (Misspecified Setting)
# =====================

# Set up parameters for plotting
x_grid = np.arange(0, 251, 50)
n_arms = len(x_grid)
T = 100
reps = 30
px_values = [0.3, 0.5, 0.7]

color_map = {arm: plt.cm.tab10(i % 10) for i, arm in enumerate(x_grid)}  # Color for each arm
labels = {
        "eps": "Epsilon-Greedy",
        "ucb": "UCB",
        "violin": "ViOlin"
    }

# For each price and model, plot the running proportion for each algorithm in the misspecified setting
for p_x in px_values:
    for model_name in models.keys():
        if model_name == "mitscherlich":
            true_name = "quadratic_plateau"
        else:
            true_name = "mitscherlich"
        true_name_cleaned = true_name.replace('_', ' ').title()
        fit_name_cleaned = model_name.replace('_', ' ').title()
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        fig.suptitle(f"Running Proportion of Most Frequently Selected Fertilizer Over Time — $p_x$ = {p_x} \n True: {true_name_cleaned} - Fit: {fit_name_cleaned}", fontsize=16, fontweight="bold")

        for ax, alg in zip(axs, ["eps", "ucb", "violin"]):
            chosen_arms = arm_data[p_x][model_name][alg]

            # Calculate average running proportion for each arm
            running_props = np.zeros((n_arms, T))
            for a_index, arm in enumerate(x_grid):
                running_props[a_index] = np.cumsum(chosen_arms == arm) / np.arange(1, T + 1)
            
            line_styles = ['-', '--', '-.', ':', (0, (3, 1, 1, 1)), (0, (5, 1))]

            # Plot running proportion for each arm
            for a_index, arm in enumerate(x_grid):
                label = f"{arm}"
                linestyle = line_styles[a_index % len(line_styles)]  # Cycle through line styles
                ax.plot(
                    running_props[a_index], 
                    label=label, 
                    linestyle=linestyle, 
                    color=color_map[arm], 
                    linewidth=2
                )

            ax.set_title(f"{labels[alg]}", fontweight="bold")
            ax.set_xlabel("Round")
            ax.set_ylabel("Proportion selected up to round")
            ax.set_ylim(0, 1)
            ax.grid(True)
            ax.legend(title='Fertilizer Rate (lb N/ac)', fontsize=8, ncol=2)

        plt.tight_layout()
        # Save figures to different folders based on scenario
        if (p_x == 0.5) and (model_name == "quadratic_plateau"):
            out_dir = f"{out_folder_name}/misspecified"
            os.makedirs(out_dir, exist_ok=True)
            plt.savefig(f"{out_dir}/running_proportion_fit-{model_name}_true-{true_name}_px_{p_x}.png", dpi=300, bbox_inches='tight')
        else:
            out_dir = f"{out_folder_name}/appendix/misspecified/running_proportion"
            os.makedirs(out_dir, exist_ok=True)
            plt.savefig(f"{out_dir}/running_proportion_fit-{model_name}_true-{true_name}_px_{p_x}.png", dpi=300, bbox_inches='tight')
        plt.close()
        # plt.show()

### Plotting arm selection over time

In [ ]:
# =====================
# Plotting: Most Frequently Selected Fertilizer Rate Over Time (Misspecified Setting)
# =====================

# Set up parameters for plotting
x_grid = np.arange(0, 251, 50)
px_values = [0.3, 0.5, 0.7]
colors = {"eps": "tab:blue", "ucb": "tab:orange", "violin": "tab:green"}
linestyles = {"eps": "--", "ucb": "-", "violin": ":"}
labels = {"eps": "Epsilon-Greedy", "ucb": "UCB", "violin": "ViOlin"}

# For each model, plot the most frequently selected arm for each price and algorithm in the misspecified setting
for model_name in models.keys():
    if model_name == "mitscherlich":
        true_name = "quadratic_plateau"
    else:
        true_name = "mitscherlich"
    true_name_cleaned = true_name.replace('_', ' ').title()
    fit_name_cleaned = model_name.replace('_', ' ').title()
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f"Most Frequently Selected Fertilizer Rate Over Time By Price \n True: {true_name_cleaned} - Fit: {fit_name_cleaned}", fontsize=16, fontweight="bold")

    for ax, p_x in zip(axs, px_values):
        for alg in ["eps", "ucb", "violin"]:
            arm = arm_data[p_x][model_name][alg]
            ax.plot(
                arm,
                label=f"{labels[alg]}",
                color=colors[alg],
                linestyle=linestyles[alg],
                linewidth=2
            )

        ax.set_title(f"$p_x$ = {p_x}", fontweight="bold")
        ax.set_xlabel("Round")
        ax.set_ylabel("Most Frequently Selected Fertilizer Rate")
        ax.set_ylim(0, arm_data[p_x][model_name]["eps"].max() + 10)
        ax.grid(True)
        ax.legend(title="Bandit Algorithm", fontsize=9)

    plt.tight_layout()
    # Save figures to different folders based on scenario
    if (model_name == "quadratic_plateau"):
        out_dir = f"{out_folder_name}/misspecified"
        os.makedirs(out_dir, exist_ok=True)
        plt.savefig(f"{out_dir}/avg_arm_fit-{model_name}_true-{true_name}.png", dpi=300, bbox_inches='tight')
    else:
        out_dir = f"{out_folder_name}/appendix/misspecified/mode_arm"
        os.makedirs(out_dir, exist_ok=True)
        plt.savefig(f"{out_dir}/mode_arm_fit-{model_name}_true-{true_name}.png", dpi=300, bbox_inches='tight')
    plt.close()
    # plt.show()

### Plotting Parameters over time

In [ ]:
# =====================
# Plotting: Parameters Over Time (Misspecified Setting)
# =====================

# For each price and model, plot the evolution of parameter estimates for each algorithm in the misspecified setting
px_values = [0.3, 0.5, 0.7]
labels = {"eps": "Epsilon-Greedy", "ucb": "UCB", "violin": "ViOlin"}

for p_x in px_values:
    for model_name, model_info in models.items():
        if model_name == "mitscherlich":
            true_name = "quadratic_plateau"
        else:
            true_name = "mitscherlich"
        true_name_cleaned = true_name.replace('_', ' ').title()
        fit_name_cleaned = model_name.replace('_', ' ').title()
        param_names = list(model_info["params0"].keys())  # Parameter names for the model
        num_params = len(param_names)

        # Choose subplot layout based on number of parameters
        if num_params == 4:
            fig, axs = plt.subplots(2, 2, figsize=(12, 10))
        elif num_params == 3:
            fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        else:
            print(f"Unexpected number of parameters ({num_params}) for model {model_name}.")
        axs = axs.ravel()

        fig.suptitle(f"Parameters Over Time — $p_x$ = {p_x} \n True: {true_name_cleaned} - Fit: {fit_name_cleaned}", fontsize=16, fontweight="bold")

        # Plot parameter evolution for each algorithm
        for alg, linestyle in zip(["eps", "ucb", "violin"], ["--", "-.", ":"]):
            param_array = param_data[p_x][model_name][alg]

            for i, param_name in enumerate(param_names):
                # Plot true parameter value as a horizontal line (only once per subplot)
                axs[i].axhline(model_info["params_true"][param_name], color="red", linestyle="-", linewidth=2, label="True" if alg == "eps" else "")
                axs[i].plot(
                    np.arange(1, 101), 
                    param_array[:, i],
                    label=f"{labels[alg]}",
                    linestyle=linestyle,
                    linewidth=2
                )
                axs[i].set_ylabel(param_name)
                axs[i].set_xlabel("Round")
                axs[i].grid(True)
        
        for ax in axs[:num_params]:
            ax.legend(title="Bandit Algorithm")

        plt.tight_layout()
        # Save figures to different folders based on scenario
        if (p_x == 0.5):
            out_dir = f"{out_folder_name}/misspecified"
            os.makedirs(out_dir, exist_ok=True)
            plt.savefig(f"{out_dir}/params_fit-{model_name}_true-{true_name}_px_{p_x}.png", dpi=300, bbox_inches='tight')
        else:
            out_dir = f"{out_folder_name}/appendix/misspecified/params"
            os.makedirs(out_dir, exist_ok=True)
            plt.savefig(f"{out_dir}/params_fit-{model_name}_true-{true_name}_px_{p_x}.png", dpi=300, bbox_inches='tight')
        plt.close()
        # plt.show()